In [3]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/753.3 kB ? eta -:--:--
   --------------------------------------- 753.3/753.3 kB 15.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
!pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable


In [43]:
import json
import requests
import re
from collections import Counter
from openai import OpenAI

# STEP 0: OpenAI API 키 불러오기
with open(r"C:\Users\SCK\Desktop\openai api key.txt", "r") as f: ## <<<< openAI API 키를 TXT파일로 저장하고 주소 설정하면 됩니다.
    openai_api_key = f.read().strip()

client = OpenAI(api_key=openai_api_key)

# STEP 1: 감정 JSON 로드 및 주요 감정 추출
with open(r"C:\Users\SCK\Desktop\test\prediction_results.json", "r") as f:
    data = json.load(f)

emotions = [item["predicted_label"] for item in data]
most_common_emotion = Counter(emotions).most_common(1)[0][0]

# STEP 2: 프롬프트 생성
prompt = f"""
한 사람이 '{most_common_emotion}' 감정을 느끼고 있습니다.
이 감정 상태에 적합한 쇼핑 카테고리를 사용자에게 말하듯 자연스럽게 추천해주세요.
어울리는 쇼핑 카테고리 3개를 추천하되, 각 카테고리는 반드시 실제 쇼핑몰에서 검색 가능한 구체적인 명사로, <[카테고리명]> 형태로 묶어 출력해 주세요.
"""

# STEP 3: OpenAI Chat API 호출
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a shopping assistant."},
        {"role": "user", "content": prompt}
    ]
)

gpt_reply = response.choices[0].message.content.strip()
print(f"\n📢 GPT 응답:\n{gpt_reply}")

# STEP 4: <[카테고리명]> 형식으로 감싼 부분만 추출
categories = re.findall(r"<\[(.*?)\]>", gpt_reply)
print(f"\n🎯 추출된 카테고리 리스트: {categories}")

# 이후 네이버 쇼핑 API에 categories 리스트 넘기면 됨



📢 GPT 응답:
패닉 상태를 달래줄 수 있는 다양한 상품들이 있습니다. 아래는 그 중에서도 한사람이 고르기 좋은 카테고리 3가지입니다:
<[편안한 의류]>
<[힐링 캔들]>
<[쌀쌀한 티마그 긱오운돌]>

🎯 추출된 카테고리 리스트: ['편안한 의류', '힐링 캔들', '쌀쌀한 티마그 긱오운돌']


In [45]:
with open(r"C:\Users\SCK\Desktop\naverapi.txt", "r") as f: ## <<< 마찬가지로 NAVER API 신청하고 ID랑 키값을 2줄로 적으면 됩니다.
    lines = f.readlines()
    naver_client_id = lines[0].strip()            
    naver_client_secret = lines[1].strip()

url = "https://openapi.naver.com/v1/search/shop.json"
headers = {
    "X-Naver-Client-Id": naver_client_id,
    "X-Naver-Client-Secret": naver_client_secret
}



for cat in categories:
    params = {"query": cat, "display": 3}
    res = requests.get(url, headers=headers, params=params)
    items = res.json().get("items", [])
    print(f"\n🔍 [{cat}] 관련 상품:")
    for item in items:
        print(f" - {item['title']} / {item['lprice']}원")
        print(f"   🔗 {item['link']}")


🔍 [편안한 의류] 관련 상품:
 - 루즈핏 나시 롱 원피스 스퀘어넥 여름 휴양지 원피스 바스락 나일론 시원한 원피스 바캉스룩 / 29800원
   🔗 https://smartstore.naver.com/main/products/10552087640
 - 여름 체형커버 원피스 시원한 <b>편안한</b> 넉넉한 핏 체크 롱 원피스 66 77 빅사이즈 4컬러 / 34900원
   🔗 https://smartstore.naver.com/main/products/8632279637
 - 민소매 레이어드 에이라인 롱 나시 바스락 원피스 66사이즈 77사이즈 빅사이즈 하객 / 43000원
   🔗 https://smartstore.naver.com/main/products/7099254686

🔍 [힐링 캔들] 관련 상품:
 - 메리고라운드 &amp; 크리스탈애플 <b>캔들</b> (2종택1) 타닥타닥 우드심지 불멍 소이<b>캔들</b> 향초 / 10000원
   🔗 https://smartstore.naver.com/main/products/200191594
 - 로고스 시트로넬라 아로마 <b>힐링</b> 캠핑<b>캔들</b> - 감성캠핑 양초 / 4000원
   🔗 https://search.shopping.naver.com/catalog/30261289360
 - 로고스 로고스 시트로넬라 아로마 <b>힐링 캔들</b> 4pcs 74309010 / 3850원
   🔗 https://search.shopping.naver.com/catalog/54454807670

🔍 [쌀쌀한 티마그 긱오운돌] 관련 상품:
